In [1]:
from transformers import AutoModel, AutoTokenizer

In [2]:
'''
主要预训练模型:
'funnel-transformer/small'
'funnel-transformer/medium'
'funnel-transformer/large'
'funnel-transformer/xlarge'
'''
tokenizer = AutoTokenizer.from_pretrained('funnel-transformer/small')
print(tokenizer.model_input_names)  # 动态编码
print(tokenizer.all_special_tokens)

['input_ids', 'token_type_ids', 'attention_mask']
['<s>', '</s>', '<unk>', '<sep>', '<pad>', '<cls>', '<mask>']


In [3]:
model = AutoModel.from_pretrained("funnel-transformer/small")
model

FunnelModel(
  (embeddings): FunnelEmbeddings(
    (word_embeddings): Embedding(30522, 768)
    (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): FunnelEncoder(
    (attention_structure): FunnelAttentionStructure(
      (sin_dropout): Dropout(p=0.1, inplace=False)
      (cos_dropout): Dropout(p=0.1, inplace=False)
    )
    (blocks): ModuleList(
      (0): ModuleList(
        (0): FunnelLayer(
          (attention): FunnelRelMultiheadAttention(
            (hidden_dropout): Dropout(p=0.1, inplace=False)
            (attention_dropout): Dropout(p=0.1, inplace=False)
            (q_head): Linear(in_features=768, out_features=768, bias=False)
            (k_head): Linear(in_features=768, out_features=768, bias=True)
            (v_head): Linear(in_features=768, out_features=768, bias=True)
            (post_proj): Linear(in_features=768, out_features=768, bias=True)
            (layer_norm): LayerNorm((768,), 

In [4]:
model.config

FunnelConfig {
  "_name_or_path": "funnel-transformer/small",
  "activation_dropout": 0.0,
  "architectures": [
    "FunnelModel"
  ],
  "attention_dropout": 0.1,
  "attention_type": "relative_shift",
  "block_repeats": [
    1,
    1,
    1
  ],
  "block_sizes": [
    4,
    4,
    4
  ],
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "hidden_act": "gelu_new",
  "hidden_dropout": 0.1,
  "initializer_range": 0.1,
  "initializer_std": null,
  "layer_norm_eps": 1e-09,
  "max_position_embeddings": 512,
  "model_type": "funnel",
  "n_head": 12,
  "num_decoder_layers": 2,
  "pool_q_only": true,
  "pooling_type": "mean",
  "rel_attn_type": "factorized",
  "separate_cls": true,
  "transformers_version": "4.17.0",
  "truncate_seq": true,
  "type_vocab_size": 3,
  "vocab_size": 30522
}

In [5]:
for name, para in model.named_parameters():
    print(name)

embeddings.word_embeddings.weight
embeddings.layer_norm.weight
embeddings.layer_norm.bias
encoder.blocks.0.0.attention.r_w_bias
encoder.blocks.0.0.attention.r_r_bias
encoder.blocks.0.0.attention.r_kernel
encoder.blocks.0.0.attention.r_s_bias
encoder.blocks.0.0.attention.seg_embed
encoder.blocks.0.0.attention.q_head.weight
encoder.blocks.0.0.attention.k_head.weight
encoder.blocks.0.0.attention.k_head.bias
encoder.blocks.0.0.attention.v_head.weight
encoder.blocks.0.0.attention.v_head.bias
encoder.blocks.0.0.attention.post_proj.weight
encoder.blocks.0.0.attention.post_proj.bias
encoder.blocks.0.0.attention.layer_norm.weight
encoder.blocks.0.0.attention.layer_norm.bias
encoder.blocks.0.0.ffn.linear_1.weight
encoder.blocks.0.0.ffn.linear_1.bias
encoder.blocks.0.0.ffn.linear_2.weight
encoder.blocks.0.0.ffn.linear_2.bias
encoder.blocks.0.0.ffn.layer_norm.weight
encoder.blocks.0.0.ffn.layer_norm.bias
encoder.blocks.0.1.attention.r_w_bias
encoder.blocks.0.1.attention.r_r_bias
encoder.blocks.0.1

In [6]:
text = """Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length.
To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer.
Longformer’s attention mechanism is a drop-in replacement for the standard self-attention and combines a local windowed attention with a task motivated global attention.
Following prior work on long-sequence transformers, we evaluate Longformer on character-level language modeling and achieve state-of-the-art results on text8 and enwik8.
In contrast to most prior work, we also pretrain Longformer and finetune it on a variety of downstream tasks.
Our pretrained Longformer consistently outperforms RoBERTa on long document tasks and sets new state-of-the-art results on Wiki-Hop and TriviaQA."""
text = text * 5
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(encoded_input['input_ids'].shape)  # [1, 408]

Token indices sequence length is longer than the specified maximum sequence length for this model (1007 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[  101, 10938,  2121,  ..., 19062,  1012,   102]]), 'token_type_ids': tensor([[2, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([1, 1007])


In [8]:
result = model(**encoded_input)
print(result.last_hidden_state)
print(result.last_hidden_state.shape)  # [1, 408, 768]

tensor([[[-0.2550, -0.2578,  0.6413,  ..., -0.1377,  0.9703, -0.4526],
         [ 0.0523, -0.1231, -0.0771,  ...,  0.4967,  0.8560,  0.2351],
         [-0.4890,  0.0853,  0.2728,  ...,  0.2254,  0.4350,  0.0144],
         ...,
         [-0.1395, -0.4087,  0.3198,  ...,  0.0335,  0.1080, -0.7443],
         [-0.4720, -0.1379, -0.0346,  ...,  0.3574,  0.7269, -1.5495],
         [-0.2128, -0.1527, -0.2427,  ...,  0.7988,  0.5887, -0.4585]]],
       grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 1007, 768])
